# CNN-RNN

In [1]:
import tensorflow.keras as keras

In [2]:
# Módulos necesarios. Asegurarse de poder importarlos.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from pylab import rcParams
from scipy import stats
from sklearn import metrics
import seaborn as sns
from sklearn import preprocessing
import os as os
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
from time import time
from tensorflow.keras.models import load_model

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [3]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
tf.__version__

/usr/bin/python
3.6.8 (default, Aug 20 2019, 17:12:48) 
[GCC 8.3.0]
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)


'1.14.0'

In [4]:
keras.__version__

'2.2.4-tf'

# 10-FOLD

In [5]:
subjects = ['01', '02', '03', '05', '08', '09', '10', '11', '13', '14', '16', '17']
global activities
activities = [9,10,11,12,13,19,20,21,24,25,31]

In [6]:
global size
size = 26125

In [7]:
from sklearn.model_selection import KFold
S = np.arange(0,size)
kf = KFold(n_splits=10)
kf.get_n_splits(S)

print(kf)  

S_train = []
S_test = []

for train_index, test_index in kf.split(S):
    S_train.append(S[train_index])
    S_test.append(S[test_index])

KFold(n_splits=10, random_state=None, shuffle=False)


In [8]:
def perf_measure(cm1):
    temp = 0
    TruePositive = np.diag(cm1)
    FalsePositive = []
    for i in range(len(activities)):
        FalsePositive.append(sum(cm1[:,i]) - cm1[i,i])
    FalseNegative = []
    for i in range(len(activities)):
        FalseNegative.append(sum(cm1[i,:]) - cm1[i,i])
    TrueNegative = []
    for i in range(len(activities)):
        temp = np.delete(cm1, i, 0)   # delete ith row
        temp = np.delete(temp, i, 1)  # delete ith column
        TrueNegative.append(sum(sum(temp)))
    
    return(TruePositive, FalsePositive, TrueNegative, FalseNegative)

In [9]:
# Ajustado a los datos
n_time_steps = 128
n_classes = 11 # Nº de clases (el 0 está eliminado)
n_channels = 1 # Nº de canales
n_columns = 40

# Podríamos variarlo
batch_size = 128 # Tamaño del batch
learning_rate = 1e-3 # Learning rate (por defecto es 0.001)
epochs = 100 # Épocas

In [10]:
accuracies = np.zeros((10))
precision = np.zeros((10,11))
recall = np.zeros((10,11))
fScore = np.zeros((10,11))
support = np.zeros((10,11))
specifities = np.zeros((10,11))

classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
measure = ['precision', 'recall', 'f1-score', 'support']

for i in range(10):
    if(i<10):
        kfold='0'+str(i)
    else:
        kfold=str(i)
    print('##################################################################################################################################')
    print('#####################################################  K-FOLD %d  ################################################################'%(i+1))
    
    subjects = ['01', '02', '03', '05', '08', '09', '10', '11', '13', '14', '16', '17']
    
    ##  GENERACIÓN DATASET
    tr_seg = np.concatenate((np.load('./augment_2/subject_'+subjects[0]+'_seg.npy'),np.load('./augment_2/subject_'+subjects[1]+'_seg.npy'),np.load('./augment_2/subject_'+subjects[2]+'_seg.npy'),np.load('./augment_2/subject_'+subjects[3]+'_seg.npy'),
                             np.load('./augment_2/subject_'+subjects[4]+'_seg.npy'),np.load('./augment_2/subject_'+subjects[5]+'_seg.npy'),np.load('./augment_2/subject_'+subjects[6]+'_seg.npy'),np.load('./augment_2/subject_'+subjects[7]+'_seg.npy'),
                             np.load('./augment_2/subject_'+subjects[8]+'_seg.npy'),np.load('./augment_2/subject_'+subjects[9]+'_seg.npy'),np.load('./augment_2/subject_'+subjects[10]+'_seg.npy')))

    tr_lab = np.concatenate((np.load('./augment_2/subject_'+subjects[0]+'_lab.npy'),np.load('./augment_2/subject_'+subjects[1]+'_lab.npy'),np.load('./augment_2/subject_'+subjects[2]+'_lab.npy'),np.load('./augment_2/subject_'+subjects[3]+'_lab.npy'),
                             np.load('./augment_2/subject_'+subjects[4]+'_lab.npy'),np.load('./augment_2/subject_'+subjects[5]+'_lab.npy'),np.load('./augment_2/subject_'+subjects[6]+'_lab.npy'),np.load('./augment_2/subject_'+subjects[7]+'_lab.npy'),
                             np.load('./augment_2/subject_'+subjects[8]+'_lab.npy'),np.load('./augment_2/subject_'+subjects[9]+'_lab.npy'),np.load('./augment_2/subject_'+subjects[10]+'_lab.npy')))
    
    limit = 40
    tr_seg = tr_seg[:,:,:,:limit]
    ####################################################################################################################
    ## SHUFFLE DE DATOS
    np.random.seed(235)
    tr_seg = np.reshape(tr_seg[np.random.shuffle(np.arange(0,tr_seg.shape[0]))], (size,1,128,limit))
    tr_lab = np.reshape(tr_lab[np.random.shuffle(np.arange(0,tr_seg.shape[0]))], (size,11))
    
    ts_seg = tr_seg[S_test[i]]
    ts_lab = tr_lab[S_test[i]]
    
    del tr_seg, tr_lab
    
    print('Test dataset: ')
    print(ts_seg.shape, ts_lab.shape)
    ####################################################################################################################
    ## RED
    model = load_model('./04AUGFFT_RP_KFOLD_models/CNN_RNN_'+kfold+'.h5')
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=learning_rate),metrics=['categorical_accuracy'])
    ####################################################################################################################
    ## TEST
    predictions = model.predict(ts_seg, batch_size = batch_size, verbose = 0)
    
    y_pred = np.argmax(predictions,axis=-1)
    y_pred = np.expand_dims(y_pred,axis=-1)
    
    y_true = np.argmax(ts_lab,axis=-1)
    y_true = np.expand_dims(y_true,axis=-1)
    
    y_true_flat = np.ndarray.flatten(y_true)
    y_pred_flat = np.ndarray.flatten(y_pred)
    ####################################################################################################################
    ## METRICS        
    # CONFUSION MATRIX
    print('\n')
    print('Confusion matrix:')
    cm = metrics.confusion_matrix(y_true, y_pred)
    print(cm)
    
    # ACCURACY
    print('\n')
    acc = metrics.accuracy_score(y_true_flat,y_pred_flat)
    accuracies[i] = acc
    print('Total accuracy: ')
    print(acc)
    
    # CLASSIFICATION REPORT
    print('\n')
    print('Classification report:')
    reportString = metrics.classification_report(y_true_flat, y_pred_flat, digits=3)
    print(reportString)
    report = metrics.classification_report(y_true_flat, y_pred_flat, digits=3, output_dict=True)
    
    for j in range(len(classes)):
        precision[i,j] = report[classes[j]][measure[0]]
        recall[i,j] = report[classes[j]][measure[1]]
        fScore[i,j] = report[classes[j]][measure[2]]
        support[i,j] = report[classes[j]][measure[3]]
    
    
    # SPECIFITY
    print('\n')
    tp, fp, tn, fn = perf_measure(cm)
    print('True Positive: ', str(tp))
    print('False Positive: ', str(fp))
    print('True Negative: ', str(tn))
    print('False Negative: ', str(fn))
    print('\n')
    specifity = []
    print('Specifity: ')
    for j in range(len(activities)):
        specifity.append(tn[j]/(tn[j]+fp[j]))
        print('%d'%(j)+'\t'+ str(specifity[j]))
    
    for j in range(len(specifity)):
        specifities[i,j] = specifity[j]
    
    fo = open('./04AUGFFT_RP_KFOLD_models/metrics_'+kfold+'.txt', "w")
    fo.seek(0,2)
    fo.write('Accuracy: ' + str(acc))
    fo.write('\n')
    fo.write('Confusion matrix: ')
    fo.write('\n')
    fo.write(str(cm))
    fo.write('\n')
    fo.write('Specifity:')
    fo.write(str(specifity))
    fo.write('\n')
    fo.write('Report: ')
    fo.write('\n')
    fo.write(reportString)
    fo.close()
    
    
    del kfold, ts_seg, ts_lab, model, y_pred, y_true, predictions, y_true_flat, y_pred_flat, reportString, specifity, cm, report
    

##################################################################################################################################
#####################################################  K-FOLD 1  ################################################################
Test dataset: 
(2613, 1, 128, 40) (2613, 11)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Confusio

# TOTAL METRICS

In [11]:
print('Total accuracy: ' + str(sum(accuracies)/10))

prec = np.sum(precision,axis=0)/10
print('Precision per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(prec[j]))
print('Total precision: ' + str(sum(prec)/11))

rec = np.sum(recall,axis=0)/10
print('Recall per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(rec[j]))
print('Total recall: ' + str(sum(rec)/11))

fS = np.sum(fScore,axis=0)/10
print('F1-score per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(fS[j]))
print('Total F1-score: ' + str(sum(fS)/11))

spec = np.sum(specifities,axis=0)/10
print('Specifities per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(spec[j]))
print('Total specifities: ' + str(sum(spec)/11))

Total accuracy: 0.9920780711825488
Precision per class: 
0	1.0
1	0.9987234042553192
2	1.0
3	0.999163179916318
4	1.0
5	0.9612820512820512
6	1.0
7	1.0
8	0.9824137931034482
9	1.0
10	1.0
Total precision: 0.9946893116870125
Recall per class: 
0	1.0
1	0.9991452991452991
2	1.0
3	1.0
4	1.0
5	1.0
6	0.9431451612903226
7	0.9944444444444445
8	1.0
9	0.9779220779220779
10	1.0
Total recall: 0.9922415438911039
F1-score per class: 
0	1.0
1	0.9989339019189766
2	1.0
3	0.9995798319327731
4	1.0
5	0.975993640699523
6	0.9602816901408451
7	0.9971428571428571
8	0.9903591682419659
9	0.9875912408759124
10	1.0
Total F1-score: 0.9918074846320777
Specifities per class: 
0	1.0
1	0.9998738965952081
2	1.0
3	0.9999158249158249
4	1.0
5	0.9936394271272114
6	1.0
7	1.0
8	0.9978517270429655
9	1.0
10	1.0
Total specifities: 0.9992073523346555
